<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_HR_gr2/blob/main/Kulinichenko%20Alexander/%D0%A1%D1%82%D0%B0%D0%B6%D0%B8%D1%80%D0%BE%D0%B2%D0%BA%D0%B0_%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5_2_%D0%B3%D1%80%D1%83%D0%BF%D0%BF%D0%B0_01_04_2023_%D0%9A%D1%83%D0%BB%D0%B8%D0%BD%D0%B8%D1%87%D0%B5%D0%BD%D0%BA%D0%BE_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D0%B1%D0%B0%D0%B7%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Работа с массивами данных
import numpy as np 

# Работа с табличными данными
import pandas as pd

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential, Model

# Основные слои
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Embedding, Flatten, Activation, Input, concatenate
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D

# Оптимизаторы
from tensorflow.keras.optimizers import Adam, Adadelta, SGD, Adagrad, RMSprop

# Токенизатор для преобразование текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Масштабирование данных
from sklearn.preprocessing import StandardScaler

# Загрузка датасетов из облака google
import gdown

# Регулярные выражения
import re

# Отрисовка графиков
import matplotlib.pyplot as plt

# Метрики для расчета ошибок
from sklearn.metrics import mean_squared_error, mean_absolute_error

%matplotlib inline

#Изучение датасета

In [ ]:

# Загрузка файла из облака в colab
pact_file = "https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_HR/datasets/%D0%94%D0%B0%D1%82%D0%B0%D0%A1%D0%B5%D1%82%20%D0%9E%D1%86%D0%B5%D0%BD%D0%BA%D0%B0%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20581%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5.xlsx"
gdown.download(pact_file, None, quiet=True)
data_xsl = pd.read_excel(pact_file, sheet_name='Лист1')
data_xsl.head()

,Unnamed: 0,Дата отклика,ФИО,"Пол, возраст",ЗП,Ищет работу на должность,Город,Занятость,График,Опыт (),Последнее/нынешное место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Подходит - да/нет
0,0,2023-02-23 00:00:00,Bessonova Mariya,Женщина,90 000 руб.,Growth Product Manager,"Индонезия, Не готова к переезду, не готова к к...","проектная работа, частичная занятость, полная ...",удаленная работа,Опыт работы —5 лет 10 месяцев,PT MYBALI APP GROUP,Менеджер по продукту,"Высшее, 2023, Нетоло́гия, 2014, Университет Ро...",23 февраля 2023 в 16:30,да
1,1,2023-01-11 00:00:00,Ефремова Екатерина Вячеславовна,Женщина,не указана,Growth Product Manager,"Москва, Готова к переезду, готова к командировкам",полная занятость,"удаленная работа, гибкий график, полный день",Опыт работы —11 лет 11 месяцев,Банк Открытие,Product owner трайб сбережения и инвестиции,"Высшее, 2022, GeekBrains, 2010, Московский худ...",8 февраля 2023 в 09:15,да
2,2,2023-02-27 00:00:00,Фролов Артур Михайлович,"Мужчина, 33 года, родился 23 августа 1989",150 000 руб.,Growth Product Manager,"Санкт-Петербург, Готов к переезду: Москва, гот...","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",Опыт работы —7 лет 2 месяца,DIKIY LLC,Директор по маркетингу и бренд-менеджер,"Высшее, 2013, Санкт-Петербургский государствен...",28 февраля 2023 в 09:41,да
3,3,2023-01-11 00:00:00,Хозаев Расул Шагаманович,"Мужчина, 32 года, родился 7 декабря 1990",не указана,Growth Product Manager,"Москва, м. Молодежная, Хочу переехать, готов к...","стажировка, частичная занятость, полная занятость","удаленная работа, гибкий график, полный день",Опыт работы —8 лет 10 месяцев,WeInvest,Product Manager (team leader),"Магистр, 2014, Московский государственный техн...",24 января 2023 в 12:07,да
4,4,2023-01-11 00:00:00,Шадрин Олег Михайлович,"Мужчина, 26 лет, родился 21 февраля 1996",150 000 руб.,Growth Product Manager,"Пермь, Хочу переехать, готов к командировкам","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",Опыт работы —5 лет 9 месяцев,Индивидуальное предпринимательство / частная п...,Product/Project manager,"Высшее, 2021, Российская академия народного хо...",6 февраля 2023 в 11:18,да


In [ ]:
#количество строк и столбцов
data_xsl.shape

(579, 15)

In [ ]:
data_xsl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579 entries, 0 to 578
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Unnamed: 0                       579 non-null    int64 
 1   Дата отклика                     579 non-null    object
 2   ФИО                              579 non-null    object
 3   Пол, возраст                     579 non-null    object
 4   ЗП                               579 non-null    object
 5   Ищет работу на должность         579 non-null    object
 6   Город                            579 non-null    object
 7   Занятость                        579 non-null    object
 8   График                           579 non-null    object
 9   Опыт ()                          579 non-null    object
 10  Последнее/нынешное место работы  579 non-null    object
 11  Последняя/нынешняя должность     579 non-null    object
 12  Образование и ВУЗ                579

##Предварительные выводы:
0. Номер по порядку
1. дата отклика
2. ФИО
3. Пол, возраст.  ЗАМЕЧАНИЕ: Не у всех заполнен возраст и дата рождения
4. ЗП. ЗАМЕЧАНИЕ: У некоторых не указана. Есть в различной валюте
5. Ищет работу на должность. ЗАМЕЧАНИЕ: Всего 8 позиций
6. Город. ЗАМЕЧАНИЕ: У некоторых всесто города указана страна. так же в этом поле инвормация готов, неготов к переезду, командировкам
7. Занятость. ЗАМЕЧАНИЕ: Всего 20 разновидностей заполнения (из 5 возможных)типа: полная занятость, частичная занятость, проектная работа, стажировка, волонтерство
8. График.  ЗАМЕЧАНИЕ: Всего 30 вариантов заполнения ( из 5 возможных) типа вахтовый метод, удаленная работа (удаленная), гибкий график, полный день, сменный график. Также ошибки типа: "сменный графи", "удаленная работа\n\n". Из за чего увеличиваются варианты заполнения.
9. Опыт (). ЗАМЕЧАНИЕ: указывается количество лет и месяцев
10. Последнее/нынешное место работы. 
11. Последняя/нынешняя должность.
12. Образование и ВУЗ. Замечание Образование (12 вариантов есть повторяющиеся)
13. Обновление резюме.
14. Подходит - да/нет.  ЗАМЕЧАНИЕ: Да - 11, НЕТ - 489, Не запполнено - 79. По данному столбцу база разбалансировна.

В техническом задании есть ссылки на блоки “Ключевые навыки”, “Опыт работы”, “Обо мне”, которые отсутствуют в исходной базе данных. 

In [ ]:
#data_xsl_ = data_xsl.groupby(['Ищет работу на должность']).count()
#data_xsl_ = data_xsl.groupby(['ЗП']).count()
data_xsl_ = data_xsl.groupby(['Занятость']).count()
#data_xsl_ = data_xsl.groupby(['График']).count()
#data_xsl_ = data_xsl.groupby(['Опыт ()']).count()
#data_xsl_ = data_xsl.groupby(['Последняя/нынешняя должность']).count()
#data_xsl_ = data_xsl.groupby(['Подходит - да/нет']).count()
data_xsl_

,Дата отклика,ФИО,"Пол, возраст",ЗП,Ищет работу на должность,Город,График,Опыт (),Последнее/нынешное место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Подходит - да/нет
Занятость,,,,,,,,,,,,,
полная занятость,342,342,342,342,342,342,342,342,342,342,342,342,300
"полная занятость, проектная работа",8,8,8,8,8,8,8,8,8,8,8,8,7
"полная занятость, стажировка",1,1,1,1,1,1,1,1,1,1,1,1,0
"полная занятость, частичная занятость",22,22,22,22,22,22,22,22,22,22,22,22,22
"полная занятость, частичная занятость, проектная работа",14,14,14,14,14,14,14,14,14,14,14,14,11
"полная занятость, частичная занятость, проектная работа, волонтерство",1,1,1,1,1,1,1,1,1,1,1,1,1
"полная занятость, частичная занятость, проектная работа, стажировка",4,4,4,4,4,4,4,4,4,4,4,4,1
проектная работа,2,2,2,2,2,2,2,2,2,2,2,2,0
"проектная работа, волонтерство, частичная занятость, полная занятость",1,1,1,1,1,1,1,1,1,1,1,1,1


In [ ]:
data_xsl.describe(include=['O'])

,Дата отклика,ФИО,"Пол, возраст",ЗП,Ищет работу на должность,Город,Занятость,График,Опыт (),Последнее/нынешное место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Подходит - да/нет
count,579,579,579,579,579,579,579,579,579,579,579,579,579,500
unique,89,503,443,44,8,204,20,29,252,483,340,487,547,2
top,2023-02-22 00:00:00,фио скрыто соискателем,женщина,не указана,growth product manager,"уфа, не готова к переезду, не готова к команди...",полная занятость,полный день,опыт работы —6 лет 9 месяцев,индивидуальное предпринимательство / частная п...,product manager,среднее образование,31 января 2023 в 09:52,нет
freq,48,7,41,288,198,71,342,202,10,12,65,15,3,489


# Проверка базы

In [ ]:
data_xsl = data_xsl.drop(data_xsl.columns[0],axis = 1) # Убираем первый столбец с продублированными индексами. Он нам не нужен

In [ ]:

# Замена концов строк на пробелы, удаление символа с кодом 0xA0
# обрезка краевых пробелов, приведение к нижнему регистру
def purify(x):
    if isinstance(x, str):                # Если значение - строка:
        x = x.replace('\n', ' ').replace('\xa0', '').strip().lower()
    return x

data_xsl = data_xsl.applymap(purify)
data_xsl.head()              

,Дата отклика,ФИО,"Пол, возраст",ЗП,Ищет работу на должность,Город,Занятость,График,Опыт (),Последнее/нынешное место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Подходит - да/нет
0,2023-02-23 00:00:00,bessonova mariya,женщина,90 000 руб.,growth product manager,"индонезия, не готова к переезду, не готова к к...","проектная работа, частичная занятость, полная ...",удаленная работа,опыт работы —5 лет 10 месяцев,pt mybali app group,менеджер по продукту,"высшее, 2023, нетоло́гия, 2014, университет ро...",23 февраля 2023 в 16:30,да
1,2023-01-11 00:00:00,ефремова екатерина вячеславовна,женщина,не указана,growth product manager,"москва, готова к переезду, готова к командировкам",полная занятость,"удаленная работа, гибкий график, полный день",опыт работы —11 лет 11 месяцев,банк открытие,product owner трайб сбережения и инвестиции,"высшее, 2022, geekbrains, 2010, московский худ...",8 февраля 2023 в 09:15,да
2,2023-02-27 00:00:00,фролов артур михайлович,"мужчина, 33 года, родился 23 августа 1989",150 000 руб.,growth product manager,"санкт-петербург, готов к переезду: москва, гот...","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",опыт работы —7 лет 2 месяца,dikiy llc,директор по маркетингу и бренд-менеджер,"высшее, 2013, санкт-петербургский государствен...",28 февраля 2023 в 09:41,да
3,2023-01-11 00:00:00,хозаев расул шагаманович,"мужчина, 32 года, родился 7 декабря 1990",не указана,growth product manager,"москва, м. молодежная, хочу переехать, готов к...","стажировка, частичная занятость, полная занятость","удаленная работа, гибкий график, полный день",опыт работы —8 лет 10 месяцев,weinvest,product manager (team leader),"магистр, 2014, московский государственный техн...",24 января 2023 в 12:07,да
4,2023-01-11 00:00:00,шадрин олег михайлович,"мужчина, 26 лет, родился 21 февраля 1996",150 000 руб.,growth product manager,"пермь, хочу переехать, готов к командировкам","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",опыт работы —5 лет 9 месяцев,индивидуальное предпринимательство / частная п...,product/project manager,"высшее, 2021, российская академия народного хо...",6 февраля 2023 в 11:18,да


In [ ]:
# Разбор значений пола (вариант от елены соколовой измененный для проверки если нет указания на мужчина это или женщина)
def extract_gender(arg):
    if 'женщина' in arg.lower():
        return 0
    elif 'мужчина' in arg.lower():
        return 1
    else:
        return "2"    # указатель что не указано мужчина или женщина 

columns =['Пол', 'возраст']
data_clean = pd.DataFrame(data=None, index=None, columns=columns, dtype=None)
data_clean['Пол'] = data_xsl['Пол, возраст'].apply(extract_gender)

#data_clean
data_clean['Пол'].value_counts()  
#Если нет двоек то все хорошо


1    302
0    277
Name: Пол, dtype: int64

In [ ]:
# Разбор значений возраста

base_update_year = 2023

def extract_sex_age_years(arg):
    # Ожидается, что значение содержит "мужчина" или "женщина"
    # Если "мужчина" - результат 1., иначе 0.
    #sex = 1. if 'муж' in arg else 0.

    try:
        # Выделение года и вычисление возраста
        years = base_update_year - int(re.search(r'\d{4}', arg)[0])

    except (IndexError, TypeError, ValueError):
        # В случае ошибки год равен 0
        years = 0

    return years

#columns =['Пол', 'возраст']
#data_clean = pd.DataFrame(data=None, index=None, columns=columns, dtype=None)
data_clean['возраст'] = data_xsl['Пол, возраст'].apply(extract_sex_age_years)

#data_clean
#data_clean['возраст'].value_counts()
data_clean.agg(['sum', 'mean', 'min', 'max'])

,Пол,возраст
sum,302.000000,16892.000000
mean,0.521589,29.174439
min,0.000000,0.000000
max,1.000000,60.000000


In [ ]:
# Разбор значений возраста (2 вариает)
def extract_age(arg):
    year_key_words = ['год', 'лет', 'года']
    args_splited = arg.split(',')[:2]
    if len(args_splited) > 1:
        year = int(re.findall(r'\d+', args_splited[1])[0])
    else:
        year = 0
    return year

    
data_clean['возраст2'] = data_xsl['Пол, возраст'].apply(extract_age)
#data_clean['возраст2'].value_counts()
data_clean.agg(['sum', 'mean', 'min', 'max'])
# !!! Подсчет возраста отличается

,Пол,возраст,возраст2
sum,302.000000,16892.000000,16475.000000
mean,0.521589,29.174439,28.454231
min,0.000000,0.000000,0.000000
max,1.000000,60.000000,60.000000


In [ ]:
# Курсы валют для зарплат
currency_rate = {'руб'    : 1.,
                 'usd'    : 65.,
                 'kzt'    : 0.17,
                 'грн'    : 2.6,
                 'белруб' : 30.5,
                 'eur'    : 70.,
                 'kgs'    : 0.9,
                 'сум'    : 0.007,
                 'azn'    : 37.5,
                 'нет'    : 0.
                }

# Разбор значения зарплаты
def extract_salary(arg):
    num = arg
    # Сначала получаем чистое число, убираем лишние знаки
    if (type(num) == str):
        num = re.sub(' ','',num)
        num = re.sub('[а-яА-ЯёЁ]','',num)
        num = re.sub('[a-zA-Z]','',num)
        num = num.replace('.','')
        try:
            # Выделение числа и преобразование к float
            num = float(re.search(r'\d+', num)[0])

            # Поиск символа валюты в строке, и, если найдено,
            # приведение к рублю по курсу валюты
            for currency, rate in currency_rate.items():
                if currency in arg :
                    num *= rate
                    break
                elif currency == "нет":
                    print("Неизвестная валюта:", arg)   

        except TypeError:
            # Если не получилось выделить число - вернуть 0
            num = 0.

    return round(num / 1000, 0)                  # В тысячах рублей

data_clean['ЗП'] = data_xsl['ЗП'].apply(extract_salary) 
#data_clean
#data_clean['ЗП'].value_counts()
data_clean.agg(['sum', 'mean', 'min', 'max'])   

,Пол,возраст,возраст2,ЗП
sum,302.000000,16892.000000,16475.000000,24782.000000
mean,0.521589,29.174439,28.454231,42.801382
min,0.000000,0.000000,0.000000,0.000000
max,1.000000,60.000000,60.000000,350.000000


In [ ]:
#Находим все варианты валют столбце ЗП
unique_currency = []

i=0
for i in range(data_xsl.shape[0]): # По всему столбцу резюме
    if type(data_xsl.loc[i, 'ЗП'])!=float:
        values = data_xsl.loc[i, 'ЗП']
        values = values.split(",")
        for value in values:
            value = re.sub(' ','',value)
            value = re.sub('[0-9]','',value)
            value = value.strip()
            if value not in unique_currency:
                unique_currency.append(value)

print(unique_currency)
print(len(unique_currency))	



['руб.', 'неуказана', 'kzt', 'usd', 'eur', 'руб', 'kgs']
7


In [ ]:
#Находим все варианты в столбце Ищет работу на должность
unique_job_title = []

i=0
for i in range(data_xsl.shape[0]): # По всему столбцу резюме
    if type(data_xsl.loc[i, 'Ищет работу на должность'])!=float:
        values = data_xsl.loc[i, 'Ищет работу на должность']
        values = values.split(",")
        for value in values:
            value = value.strip()
            if value not in unique_job_title:
                unique_job_title.append(value)

print(unique_job_title)
print(len(unique_job_title))	

['growth product manager', 'product manager в growth hacking команду', 'директор медицинской клиники', 'hr-специалист', 'product manager в команду финансового департамента', 'product manager в команду hr', 'project manager', 'product marketing manager в growth hacking команду']
8


In [ ]:
#Находим все варианты в столбце Занятость
unique_employment = []

i=0
for i in range(data_xsl.shape[0]): # По всему столбцу резюме
    if type(data_xsl.loc[i, 'Занятость'])!=float:
        values = data_xsl.loc[i, 'Занятость']
        values = values.split(",")
        for value in values:
            value = value.strip()
            if value not in unique_employment:
                unique_employment.append(value)

print(unique_employment)
print(len(unique_employment))

['проектная работа', 'частичная занятость', 'полная занятость', 'стажировка', 'волонтерство', 'неполный рабочий день', 'полный рабочий день']
7


In [ ]:
#Находим все варианты в столбце График
unique_schedule = []

i=0
for i in range(data_xsl.shape[0]): # По всему столбцу резюме
    if type(data_xsl.loc[i, 'График'])!=float:
        values = data_xsl.loc[i, 'График']
        values = values.split(",")
        for value in values:
            value = value.strip()
            if value not in unique_schedule:
                unique_schedule.append(value)

print(unique_schedule)
print(len(unique_schedule))

['удаленная работа', 'гибкий график', 'полный день', 'сменный график', 'вахтовый метод', 'полный рабочий день', 'удаленная', 'сменный графи']
8


In [ ]:
#Находим все варианты образования в столбце Образование и ВУЗ 
unique_education = []

i=0
for i in range(data_xsl.shape[0]): # По всему столбцу резюме
    if type(data_xsl.loc[i, 'Образование и ВУЗ'])!=float:
        values = data_xsl.loc[i, 'Образование и ВУЗ']
        values = values.split(",")[0]
        values = values.strip()
        if values not in unique_education:
            unique_education.append(values)

print(unique_education)
print(len(unique_education))



['высшее', 'магистр', 'среднее специальное', 'бакалавр', 'неоконченное высшее', 'доктор наук', 'кандидат наук', 'среднее образование', 'среднее специальное образование', 'незаконченное высшее образование', 'высшее образование', 'среднее']
12


In [ ]:
#Находим все варианты городов, стран (первое вхождение) в столбце Город
unique_city = []
millionCities_lst = "Москва, Санкт-Петербург, Новосибирск, Екатеринбург, Нижний Новгород, Казань, Челябинск, Омск, Самара, Ростов-на-Дону, Уфа, Красноярск, Пермь, Воронеж, Волгоград,"
i=0
for i in range(data_xsl.shape[0]): # По всем резюме
    if type(data_xsl.loc[i, 'Город'])!=float:
        values = data_xsl.loc[i, 'Город']
        values = values.split(",")[0]
        values = values.strip()
        if values not in millionCities_lst.lower():
           if values not in unique_city:
                unique_city.append(values)

print(unique_city)
print(len(unique_city))

['индонезия', 'энгальс', 'октябрьский (республика башкортостан)', 'ижевск', 'кишинёв', 'таиланд', 'алматы', 'армения', 'астана', 'турция', 'саратов', 'краснодар', 'тбилиси', 'тюмень', 'кемерово', 'тула', 'германия', 'оаэ', 'бишкек', 'израиль', 'караганда', 'брянск', 'батуми', 'ярославль', 'ташкент', 'минск', 'киров (кировская область)', 'сочи', 'томск', 'магнитогорск', 'индия', 'кострома', 'иркутск', 'сербия', 'набережные челны', 'щелково', 'ульяновск', 'стерлитамак', 'саров', 'тольятти', 'кумертау', 'актау', 'йошкар-ола', 'астрахань', 'мурманск']
45


In [ ]:
#Находим все варианты командировок, перездов (второе вхождение) в столбце Город
unique_business_trips = []

i=0
for i in range(data_xsl.shape[0]): # По всему столбцу резюме
    if type(data_xsl.loc[i, 'Город'])!=float:
        values = data_xsl.loc[i, 'Город']
        values = values.split(",")[1:]
        for value in values:
            value = value.strip()
            if value not in unique_business_trips:
                unique_business_trips.append(value)

print(unique_business_trips)
print(len(unique_business_trips))

['не готова к переезду', 'не готова к командировкам', 'готова к переезду', 'готова к командировкам', 'готов к переезду: москва', 'готов к командировкам', 'м. молодежная', 'хочу переехать', 'готова к редким командировкам', 'не готов к переезду', 'готов к переезду', 'готова к переезду: москва', 'россия', 'астраханская область', 'санкт-петербург', 'екатеринбург', 'краснодар', 'владивосток', 'казань', 'сахалинская область', 'новый уренгой', 'самара', 'тюмень', 'турция', 'готов к редким командировкам', 'не готов к командировкам', 'хочу переехать: сша', 'канада', 'португалия', 'эстония', 'хочу переехать: казань', 'иннополис', 'м. пионерская', 'м. улица дыбенко', 'м. южная', 'м. мякинино', 'м. спортивная', 'готов к переезду: кыргызстан', 'узбекистан', 'украина', 'казахстан', 'азербайджан', 'другие регионы', 'грузия', 'готов к переезду: тбилиси', 'баку', 'астана', 'уфа', 'м. комсомольская', 'м. новокосино', 'готов к переезду: ярославль', 'м. селигерская', 'хочу переехать: португалия', 'хорвати